# 🏆 Tydzień 4, Dzień 6: Projekt - Risk Manager Dashboard

To finałowy projekt modułu Quant. Zbudujemy automat do raportowania ryzyka.

**Cele na dziś:**
1.  **Portfolio Class:** Zamknięcie logiki finansowej w obiekcie Pythonowym.
2.  **Agregacja Danych:** Obliczenie krzywej kapitału (Equity Curve) dla całego portfela ważonego.
3.  **Wizualizacja:** Generowanie wykresów (Drawdown, Korelacja) i zapisywanie ich do plików.
4.  **Raportowanie:** Sklejenie statystyk i obrazków w jeden plik PDF.

---
### 🛠️ 1. Instalacja i Setup

Używamy `reportlab` do PDF i standardowego stacku do obliczeń.

In [ ]:
!uv add pandas numpy matplotlib seaborn yfinance scipy reportlab

### ⚙️ 2. Klasa `PortfolioAnalyzer`

To jest serce naszego systemu. Klasa będzie odpowiedzialna za:
1.  Pobranie danych.
2.  Obliczenie wyników portfela (ważona suma zwrotów).
3.  Wyliczenie metryk (Sharpe, VaR).
4.  Narysowanie wykresów.

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import os

sns.set_theme(style="whitegrid")

class PortfolioAnalyzer:
    def __init__(self, portfolio_dict, start_date="2020-01-01"):
        """
        portfolio_dict: słownik {ticker: waga}, np. {'SPY': 0.6, 'BTC-USD': 0.4}
        """
        self.weights = pd.Series(portfolio_dict)
        self.weights /= self.weights.sum() # Normalizacja do 100%
        self.tickers = list(portfolio_dict.keys())
        self.start_date = start_date
        self.data = None
        self.returns = None
        self.portfolio_returns = None
        
        # Ścieżki
        current_dir = os.getcwd()
        self.project_root = os.path.abspath(os.path.join(current_dir, "../../"))
        self.data_dir = os.path.join(self.project_root, "data")
        self.report_dir = os.path.join(self.data_dir, "reports")
        os.makedirs(self.report_dir, exist_ok=True)

    def load_data(self):
        print(f"📥 Pobieranie danych dla: {self.tickers}...")
        df = yf.download(self.tickers, start=self.start_date, progress=False, auto_adjust=True)
        
        if isinstance(df.columns, pd.MultiIndex):
            df = df['Close']
            
        self.data = df.dropna()
        # Log Returns dla poszczególnych aktywów
        self.returns = np.log(self.data / self.data.shift(1)).dropna()
        
        # Ważony zwrot portfela (Suma iloczynów: zwrot_aktywa * waga)
        # To jest uproszczenie dla Log Returns (dla małych zmian działa dobrze)
        self.portfolio_returns = (self.returns * self.weights).sum(axis=1)
        
        print("✅ Dane załadowane i przeliczone.")

    def calculate_metrics(self):
        """Liczy metryki dla CAŁEGO portfela."""
        r = self.portfolio_returns
        trading_days = 252
        
        metrics = {}
        metrics['Total Return'] = (r.sum()) # Log return sum = total return
        metrics['Annual Return'] = r.mean() * trading_days
        metrics['Volatility'] = r.std() * np.sqrt(trading_days)
        metrics['Sharpe'] = (metrics['Annual Return'] - 0.04) / metrics['Volatility']
        
        # VaR 95% (Historyczny)
        metrics['VaR 95%'] = r.quantile(0.05)
        
        # Max Drawdown
        wealth_index = (1 + r).cumprod()
        peak = wealth_index.cummax()
        drawdown = (wealth_index - peak) / peak
        metrics['Max Drawdown'] = drawdown.min()
        
        return metrics

    def generate_plots(self):
        """Tworzy wykresy i zapisuje je jako PNG do raportu."""
        print("📊 Generowanie wykresów...")
        
        # 1. Equity Curve (Krzywa Kapitału)
        plt.figure(figsize=(10, 5))
        cumulative = (1 + self.portfolio_returns).cumprod()
        plt.plot(cumulative, label="Portfolio Value", color="green")
        plt.title("Wzrost wartości portfela (Equity Curve)")
        plt.legend()
        plt.savefig(os.path.join(self.report_dir, "plot_equity.png"))
        plt.close()
        
        # 2. Drawdown (Pod wodą)
        plt.figure(figsize=(10, 5))
        peak = cumulative.cummax()
        dd = (cumulative - peak) / peak
        plt.fill_between(dd.index, dd, 0, color='red', alpha=0.3)
        plt.plot(dd, color='red')
        plt.title("Underwater Plot (Drawdown)")
        plt.savefig(os.path.join(self.report_dir, "plot_drawdown.png"))
        plt.close()
        
        # 3. Korelacja Składników (Heatmap)
        plt.figure(figsize=(8, 6))
        sns.heatmap(self.returns.corr(), annot=True, cmap="coolwarm", vmin=-1, vmax=1)
        plt.title("Korelacja Składników Portfela")
        plt.savefig(os.path.join(self.report_dir, "plot_corr.png"))
        plt.close()
        
        print("✅ Wykresy zapisane w data/reports/")

### 📄 3. Generator PDF (`PDFReport`)

Teraz klasa, która weźmie te liczby i obrazki, i ułoży je w ładny dokument PDF.

In [2]:
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4

class PDFReport:
    def __init__(self, metrics, report_dir):
        self.metrics = metrics
        self.report_dir = report_dir
        self.filename = os.path.join(report_dir, "Risk_Dashboard.pdf")
        
    def create(self):
        c = canvas.Canvas(self.filename, pagesize=A4)
        width, height = A4
        
        # Tytuł
        c.setFont("Helvetica-Bold", 24)
        c.drawString(50, height - 50, "Risk Manager Dashboard")
        c.setFont("Helvetica", 12)
        c.drawString(50, height - 70, "Generated by Financial AI Architect")
        
        # Tabela Metryk
        y = height - 120
        c.setFont("Helvetica-Bold", 14)
        c.drawString(50, y, "Key Performance Indicators (KPIs)")
        y -= 30
        
        c.setFont("Helvetica", 12)
        for key, value in self.metrics.items():
            # Formatowanie procentowe lub liczbowe
            if "Sharpe" in key:
                text = f"{key}: {value:.2f}"
            else:
                text = f"{key}: {value:.2%}"
            
            c.drawString(70, y, text)
            y -= 20
            
        # Wstawianie Wykresów
        # (X, Y, Width, Height) - Y liczymy od dołu strony!
        
        # 1. Equity Curve
        c.drawImage(os.path.join(self.report_dir, "plot_equity.png"), 50, 400, width=500, height=250)
        
        # 2. Drawdown
        c.drawImage(os.path.join(self.report_dir, "plot_drawdown.png"), 50, 130, width=500, height=250)
        
        c.showPage() # Nowa strona
        
        # 3. Korelacja (na drugiej stronie)
        c.setFont("Helvetica-Bold", 16)
        c.drawString(50, height - 50, "Asset Correlation Matrix")
        c.drawImage(os.path.join(self.report_dir, "plot_corr.png"), 50, 400, width=400, height=300)
        
        c.save()
        print(f"✅ Raport PDF wygenerowany: {self.filename}")

### 🎮 4. Uruchomienie Systemu

Zdefiniujmy nasz portfel marzeń i wygenerujmy raport.
Portfel:
*   40% S&P 500 (SPY)
*   20% Złoto (GLD)
*   20% Obligacje (TLT)
*   10% Nvidia (NVDA)
*   10% Bitcoin (BTC-USD)

In [3]:
# 1. Konfiguracja Portfela
my_portfolio = {
    "SPY": 0.4,
    "GLD": 0.2,
    "TLT": 0.2,
    "NVDA": 0.1,
    "BTC-USD": 0.1
}

# 2. Analiza
analyzer = PortfolioAnalyzer(my_portfolio)
analyzer.load_data()
metrics = analyzer.calculate_metrics()
analyzer.generate_plots()

# Wyświetl metryki w notebooku dla podglądu
print("\n--- SZYBKI PODGLĄD WYNIKÓW ---")
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")

# 3. Generowanie PDF
report = PDFReport(metrics, analyzer.report_dir)
report.create()

📥 Pobieranie danych dla: ['SPY', 'GLD', 'TLT', 'NVDA', 'BTC-USD']...
✅ Dane załadowane i przeliczone.
📊 Generowanie wykresów...
✅ Wykresy zapisane w data/reports/

--- SZYBKI PODGLĄD WYNIKÓW ---
Total Return: 1.0804
Annual Return: 0.1822
Volatility: 0.1718
Sharpe: 0.8278
VaR 95%: -0.0156
Max Drawdown: -0.3626
✅ Raport PDF wygenerowany: c:\Users\takze\OneDrive\Pulpit\project\financial-ai-engineering\data\reports\Risk_Dashboard.pdf


## 🧠 Zadanie Domowe: Optymalizacja przed Raportem

Obecnie wagi wpisaliśmy "na czuja" (40/20/20/10/10).
Twoim zadaniem jest połączenie wczorajszej lekcji z dzisiejszą.

**Zadanie:**
1. Stwórz nową komórkę poniżej.
2. Użyj `scipy.optimize` (kod z Dnia 5), aby znaleźć wagi dla portfela **Max Sharpe Ratio** (dla tych samych aktywów).
3. Przekaż te *zoptymalizowane* wagi do `PortfolioAnalyzer`.
4. Wygeneruj raport PDF dla portfela optymalnego.
5. Porównaj (wzrokowo) wyniki - czy Sharpe Ratio w raporcie jest wyższe niż w naszym ręcznym portfelu? (Powinno być!).

In [5]:
import scipy.optimize as sco

# --- ROZWIĄZANIE ZADANIA DOMOWEGO: OPTYMALIZACJA PRZED RAPORTEM ---

print("\n🧠 ROZPOCZYNAM OPTYMALIZACJĘ PORTFELA (MAX SHARPE)...")

# 1. Pobieramy dane (korzystamy z istniejącej instancji analyzer, która ma już self.returns)
returns = analyzer.returns
mean_returns = returns.mean()
cov_matrix = returns.cov() * 252
num_assets = len(analyzer.tickers)
risk_free_rate = 0.04

# 2. Definicja funkcji celu (Negative Sharpe)
def portfolio_volatility(weights):
    return np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

def neg_sharpe_ratio(weights):
    p_ret = np.sum(mean_returns * weights) * 252
    p_vol = portfolio_volatility(weights)
    return - (p_ret - risk_free_rate) / p_vol

# 3. Konfiguracja Solvera
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1}) # Wagi sumują się do 1
bounds = tuple((0, 1) for _ in range(num_assets)) # Brak shortów (0-100%)
init_guess = [1./num_assets] * num_assets # Start od równych wag

# 4. Uruchomienie optymalizacji
opt_result = sco.minimize(
    neg_sharpe_ratio,
    init_guess,
    method='SLSQP',
    bounds=bounds,
    constraints=constraints
)

# 5. Przygotowanie nowych wag
opt_weights = opt_result.x
# Tworzymy słownik {Ticker: Waga} dla naszej klasy PortfolioAnalyzer
opt_portfolio_dict = dict(zip(analyzer.tickers, opt_weights))

print("\n🏆 ZNALEZIONO IDEALNE WAGI:")
for t, w in opt_portfolio_dict.items():
    if w > 0.01: # Pokaż tylko istotne pozycje
        print(f"   {t}: {w:.2%}")

# 6. Generowanie Nowego Raportu
print("\n🚀 Generowanie Raportu PDF dla Portfela Optymalnego...")

# Tworzymy nową instancję analyzera z OPTYMALNYMI wagami
opt_analyzer = PortfolioAnalyzer(opt_portfolio_dict)
opt_analyzer.load_data() # Dane weźmie z cache
opt_metrics = opt_analyzer.calculate_metrics()
opt_analyzer.generate_plots() # Nadpisze wykresy w folderze reports

# Generujemy PDF z nową nazwą
opt_report = PDFReport(opt_metrics, opt_analyzer.report_dir)
# Hack: Zmieniamy nazwę pliku wyjściowego w obiekcie
opt_report.filename = os.path.join(opt_analyzer.report_dir, "Risk_Dashboard_OPTIMIZED.pdf")
opt_report.create()

# 7. Porównanie
old_sharpe = metrics['Sharpe']
new_sharpe = opt_metrics['Sharpe']

print("\n--- WERDYKT KOŃCOWY ---")
print(f"Poprzedni Sharpe: {old_sharpe:.2f}")
print(f"Nowy Sharpe:      {new_sharpe:.2f}")
if new_sharpe > old_sharpe:
    print(f"✅ Matematyka poprawiła wynik o {new_sharpe - old_sharpe:.2f} pkt!")


🧠 ROZPOCZYNAM OPTYMALIZACJĘ PORTFELA (MAX SHARPE)...

🏆 ZNALEZIONO IDEALNE WAGI:
   SPY: 7.01%
   GLD: 66.88%
   TLT: 26.11%

🚀 Generowanie Raportu PDF dla Portfela Optymalnego...
📥 Pobieranie danych dla: ['SPY', 'GLD', 'TLT', 'NVDA', 'BTC-USD']...
✅ Dane załadowane i przeliczone.
📊 Generowanie wykresów...
✅ Wykresy zapisane w data/reports/
✅ Raport PDF wygenerowany: c:\Users\takze\OneDrive\Pulpit\project\financial-ai-engineering\data\reports\Risk_Dashboard_OPTIMIZED.pdf

--- WERDYKT KOŃCOWY ---
Poprzedni Sharpe: 0.83
Nowy Sharpe:      0.56
